In [1]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
course_df_ori = pd.read_csv('./data/courses.csv').fillna('')

In [3]:
id2course_mapping = course_df_ori["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)

728

In [4]:
course_df = course_df_ori.drop(['teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df = course_df.replace('\n', '',regex=True).replace('&.;', '',regex=True).replace("--&?", "",regex=True).replace("\t", "",regex=True)


In [7]:
def clean_data(x):
        return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'groups', 'sub_groups', 'groups+subgroups', 'topics', 'will_learn', 'target_group', 'description', 'recommended_background', 'required_tools']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)

,course_name,teacher_intro,groups,sub_groups,groups+subgroups,topics,will_learn,target_group,description,recommended_background,required_tools
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,生活品味,"更多生活品味,護膚保養與化妝","生活品味_更多生活品味,生活品味_護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。,熱愛彩妝的人,"<imgsrc=""https://images.api.hahow.in/images/61...",只要你有一顆愛化妝、想變漂亮的心皆可以參加。⚠️雖然課程當中會帶到相關彩妝技巧，不過內容偏向...,所需工具為：視課程實際會用到的彩妝用品
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"藝術,設計","平面設計,繪畫與插畫","設計_平面設計,藝術_繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,每一位興趣的人都能學得來，非常容易的小技巧,"<imgsrc=""https://storage.googleapis.com/hahow-...",知道如何使用Illustrator的基本工具列,"AdobeIllustrator（必備）,camera"


In [9]:
from chinese_keybert import Chinese_Extractor
kw_extractor = Chinese_Extractor()

/home/ian/miniconda3/envs/adlhw3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-31 14:24:47.533244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 14:24:47.853822: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-31 14:24:47.963437: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable t

In [10]:
fillna['kw_name'] = ''
fillna['kw_intro'] = ''
fillna['kw_topics'] = ''
fillna['kw_will'] = ''
fillna['kw_target'] = ''
fillna['kw_desc'] = ''
fillna['kw_recommend'] = ''
fillna['kw_tool'] = ''
fillna['kw_group'] = ''
fillna['kw_sub'] = ''
fillna['kw_gs'] = ''

In [ ]:
for i in range(len(fillna)):
    s1_name = ''
    s1_intro = ''
    s1_topics = ''
    s1_will = ''
    s1_target = ''
    s1_desc = ''
    s1_recommend = ''
    s1_tool = ''
    s1_group = ''
    s1_sub = ''
    s1_gs = ''
    s_name = fillna['course_name'][i]
    s_intro = fillna['teacher_intro'][i]
    s_topics = fillna['topics'][i]
    s_group = fillna['groups'][i]
    s_sub = fillna['sub_groups'][i]
    s_gs = fillna['groups+subgroups'][i]
    s_will = fillna['will_learn'][i]
    s_target = fillna['target_group'][i]
    s_desc = fillna['description'][i]
    s_recommend = fillna['recommended_background'][i]
    s_tool = fillna['required_tools'][i]
    print(i)
    try:
        kw_name = kw_extractor.generate_keywords(s_name,top_k=3,rank_methods="mmr",diversity=0.2)
        for j in range(2):
            s1_name = s1_name + kw_name[0][j] + ','
        s1_name = s1_name + kw_name[0][2]
        fillna['kw_name'][i] = s1_name
    except:
        print('f')
    try:
        kw_intro = kw_extractor.generate_keywords(s_intro,top_k=5,rank_methods="mmr",diversity=0.15)
        for k in range(4):
            s1_intro = s1_intro + kw_intro[0][k] + ','
            s1_intro = s1_intro + kw_intro[0][4]
        fillna['kw_intro'][i] = s1_intro
    except:
        print('f')
    try:
        kw_group = kw_extractor.generate_keywords(s_group,top_k=3,rank_methods="mmr",diversity=0.2)
        for j in range(2):
            s1_group= s1_group + kw_group[0][j] + ','
        s1_group = s1_group + kw_group[0][2]
        fillna['kw_group'][i] = s1_topics
    except:
        print('f')
    try:
        kw_sub = kw_extractor.generate_keywords(s_sub,top_k=3,rank_methods="mmr",diversity=0.2)
        for j in range(2):
            s1_sub = s1_sub + kw_sub[0][j] + ','
        s1_sub = s1_sub + kw_sub[0][2]
        fillna['kw_sub'][i] = s1_sub
    except:
        print('f')
    try:
        kw_gs = kw_extractor.generate_keywords(s_gs,top_k=5,rank_methods="mmr",diversity=0.3)
        for j in range(4):
            s1_gs = s1_gs + kw_gs[0][j] + ','
        s1_gs = s1_gs + kw_gs[0][4]
        fillna['kw_gs'][i] = s1_gs
    except:
        print('f')
    try:
        kw_topics = kw_extractor.generate_keywords(s_topics,top_k=3,rank_methods="mmr",diversity=0.15)
        for j in range(2):
            s1_topics = s1_topics + kw_topics[0][j] + ','
        s1_topics = s1_topics + kw_topics[0][2]
        fillna['kw_topics'][i] = s1_topics
    except:
        print('f')
    try:
        kw_will = kw_extractor.generate_keywords(s_will,top_k=3,rank_methods="mmr",diversity=0.15)
        for j in range(2):
            s1_will = s1_will + kw_will[0][j] + ','
        s1_will = s1_will + kw_will[0][2]
        fillna['kw_will'][i] = s1_will
    except:
        print('f')
    try:
        kw_target = kw_extractor.generate_keywords(s_target,top_k=1,rank_methods="mmr",diversity=0.1)
        s1_target = s1_target + kw_target[0][0]
        fillna['kw_target'][i] = s1_target
    except:
        print('f')
    try:
        kw_desc = kw_extractor.generate_keywords(s_desc,top_k=5,rank_methods="mmr",diversity=0.3)
        for k in range(4):
            s1_desc = s1_desc + kw_desc[0][k] + ','
        s1_desc = s1_desc + kw_desc[0][4]
        fillna['kw_desc'][i] = s1_desc
    except:
        print('f')
    try:
        kw_recommend = kw_extractor.generate_keywords(s_recommend,top_k=3,rank_methods="mmr",diversity=0.3)
        for l in range(2):
            s1_recommend = s1_recommend + kw_recommend[0][l] + ','
        s1_recommend = s1_recommend + kw_recommend[0][2]
        fillna['kw_recommend'][i] = s1_recommend
    except:
        print('f')
    try:
        kw_tool = kw_extractor.generate_keywords(s_tool,top_k=3,rank_methods="mmr",diversity=0.15)
        for l in range(2):
            s1_tool = s1_tool + kw_tool[0][l] + ','
        s1_tool = s1_tool + kw_tool[0][2] + ','
        fillna['kw_tool'][i] = s1_tool
    except:
        print('f')

In [13]:
fillna.to_csv('combination1231.csv')
